In [2]:
import pandas as pd
from IPython.display import HTML

1、线图效果（关系图改造）
======

In [27]:
#历年前十大股东持股情况
#df1为季度统计摘要，data1为前十大持股明细统计
df1, data1 = ts.top10_holders(code='600000', gdtype='0') #gdtype等于1时表示流通股，默认为0 

In [28]:
df1 = df1.sort_values('quarter', ascending=True)

In [29]:
df1.tail(10)

,quarter,amount,changed,props
9,2014-12-31,910420.45,-9064.14,48.81
8,2015-03-31,912166.01,1745.56,48.90
7,2015-06-30,937424.47,25258.46,50.25
6,2015-09-30,1157483.27,220058.80,62.05
5,2015-12-31,1318904.08,161420.81,70.70
4,2016-03-15,1364799.92,45895.84,73.16
3,2016-03-18,1440154.76,75354.84,73.27
2,2016-03-31,1439599.89,-554.87,73.25
1,2016-06-30,1587371.15,147771.26,73.42
0,2016-09-30,1585130.30,-2240.85,73.32


In [11]:
qts = list(df1['quarter'])
data = list(df1['props'])

In [12]:
lgdstr = """
var axisData = """ + str(qts) + """;
var data = """ + str(data) + """;
var links = data.map(function (item, i) {
    return {
        source: i,
        target: i + 1
    };
});
links.pop();
option = {
    title: {
        text: '【浦发银行】前十大流通股东持股占比'
    },
    tooltip: {
        trigger: 'item'
    },
    xAxis: {
        type : 'category',
        boundaryGap : false,
        data : axisData
    },
    yAxis: {
        type : 'value'
    },
    series: [
        {
            type: 'line',
            layout: 'none',
            coordinateSystem: 'cartesian2d',
            symbolSize: 10,
            label: {
                normal: {
                    show: true
                }
            },
            edgeSymbol: ['circle', 'arrow'],
            edgeSymbolSize: [2, 5],
            data: data,
            links: links,
            lineStyle: {
                normal: {
                    color: '#2f4554'
                }
            }
        }
    ]
};
"""

In [13]:
headstr = """
<div id="showhere" style="width:800px; height:600px;"></div> 
<script> 
require.config({ paths:{ echarts: '//cdn.bootcss.com/echarts/3.2.3/echarts.min', } });
require(['echarts'],function(ec){
var myChart = ec.init(document.getElementById('showhere'));
"""

tailstr = """
myChart.setOption(option);
    });
</script>
"""

In [14]:
HTML(headstr + lgdstr + tailstr)

2、饼图效果
======

In [16]:
#浦发银行2016三季度前十大流通股东情况
df2, data2 = ts.top10_holders(code='600000', year=2016, quarter=3, gdtype='1')

In [17]:
data2

,quarter,name,hold,h_pro,sharetype,status
0,2016-09-30,中国移动通信集团广东有限公司,410376.37,20.00,流通A股,未变
1,2016-09-30,上海国际集团有限公司,347326.53,16.93,流通A股,未变
2,2016-09-30,富德生命人寿保险股份有限公司－传统,213802.87,10.42,流通A股,未变
3,2016-09-30,富德生命人寿保险股份有限公司－资本金,135633.26,6.61,流通A股,未变
4,2016-09-30,上海上国投资产管理有限公司,107351.62,5.23,流通A股,未变
5,2016-09-30,富德生命人寿保险股份有限公司－万能H,97725.28,4.76,流通A股,未变
6,2016-09-30,梧桐树投资平台有限责任公司,68163.95,3.32,流通A股,未变
7,2016-09-30,中国证券金融股份有限公司,57684.91,2.81,流通A股,-2240.85
8,2016-09-30,上海国鑫投资发展有限公司,41481.22,2.02,流通A股,未变
9,2016-09-30,中央汇金资产管理有限责任公司,30655.49,1.49,流通A股,未变


In [18]:
#取前十大流通股东名称
top10name = str(list(data2['name']))

In [19]:
valstrs = ''
for idx in data2.index:
    s = '{value: %s, name: \'%s\'}' % (data2.ix[idx]['h_pro'], data2.ix[idx]['name'])
    valstrs += s + ','
valstrs = valstrs[:-1]

In [20]:
datacontent = """
option = {
    tooltip: {
        trigger: 'item',
        formatter: "{a} <br/>{b}: {c} ({d}%)"
    },
    legend: {
        orient: 'vertical',
        x: 'left',
        data: """ + top10name +"""
    },
    series: [
        {
            name:'前十大流通股东：',
            type:'pie',
            radius: ['50%', '70%'],
            avoidLabelOverlap: false,
            label: {
                normal: {
                    show: false,
                    position: 'center'
                },
                emphasis: {
                    show: true,
                    textStyle: {
                        fontSize: '30',
                        fontWeight: 'bold'
                    }
                }
            },
            labelLine: {
                normal: {
                    show: false
                }
            },
            data:[
                """ + valstrs + """
            ]
        }
    ]
};

"""

In [21]:
headstr = """
<div id="mychart" style="width:800px; height:600px;"></div> 
<script> 
require.config({ paths:{ echarts: '//cdn.bootcss.com/echarts/3.2.3/echarts.min', } });
require(['echarts'],function(ec){
var myChart = ec.init(document.getElementById('mychart'));
"""

tailstr = """
myChart.setOption(option);
    });
</script>
"""

In [22]:
HTML(headstr + datacontent + tailstr)

3、K线效果演示
=========

In [23]:
#中国联通前复权数据
df = ts.get_k_data('600050', start='2016-01-01', end='2016-12-02')
datastr = ''
for idx in df.index:
    rowstr = '[\'%s\',%s,%s,%s,%s]' % (df.ix[idx]['date'], df.ix[idx]['open'], 
                                           df.ix[idx]['close'], df.ix[idx]['low'], 
                                           df.ix[idx]['high'])
    datastr += rowstr + ','
datastr = datastr[:-1]

In [24]:
#取股票名称
name = ts.get_realtime_quotes('600050')['name'][0]

In [25]:
datahead = """
<div id="chart" style="width:800px; height:600px;"></div> 
<script> 
require.config({ paths:{ echarts: '//cdn.bootcss.com/echarts/3.2.3/echarts.min', } });
require(['echarts'],function(ec){
var myChart = ec.init(document.getElementById('chart'));
"""
datavar = 'var data0 = splitData([%s]);' % datastr
funcstr = """
function splitData(rawData) {
    var categoryData = [];
    var values = []
    for (var i = 0; i < rawData.length; i++) {
        categoryData.push(rawData[i].splice(0, 1)[0]);
        values.push(rawData[i])
    }
    return {
        categoryData: categoryData,
        values: values
    };
}

function calculateMA(dayCount) {
    var result = [];
    for (var i = 0, len = data0.values.length; i < len; i++) {
        if (i < dayCount) {
            result.push('-');
            continue;
        }
        var sum = 0;
        for (var j = 0; j < dayCount; j++) {
            sum += data0.values[i - j][1];
        }
        result.push((sum / dayCount).toFixed(2));
    }
    return result;
}

option = {
    title: {
"""

namestr = 'text: \'%s\',' %name

functail = """
        left: 0
    },
    tooltip: {
        trigger: 'axis',
        axisPointer: {
            type: 'line'
        }
    },
    legend: {
        data: ['日K', 'MA5', 'MA10', 'MA20', 'MA30']
    },
    grid: {
        left: '10%',
        right: '10%',
        bottom: '15%'
    },
    xAxis: {
        type: 'category',
        data: data0.categoryData,
        scale: true,
        boundaryGap : false,
        axisLine: {onZero: false},
        splitLine: {show: false},
        splitNumber: 20,
        min: 'dataMin',
        max: 'dataMax'
    },
    yAxis: {
        scale: true,
        splitArea: {
            show: true
        }
    },
    dataZoom: [
        {
            type: 'inside',
            start: 50,
            end: 100
        },
        {
            show: true,
            type: 'slider',
            y: '90%',
            start: 50,
            end: 100
        }
    ],
    series: [
        {
            name: '日K',
            type: 'candlestick',
            data: data0.values,
            markPoint: {
                label: {
                    normal: {
                        formatter: function (param) {
                            return param != null ? Math.round(param.value) : '';
                        }
                    }
                },
                data: [
                    {
                        name: '标点',
                        coord: ['2013/5/31', 2300],
                        value: 2300,
                        itemStyle: {
                            normal: {color: 'rgb(41,60,85)'}
                        }
                    },
                    {
                        name: 'highest value',
                        type: 'max',
                        valueDim: 'highest'
                    },
                    {
                        name: 'lowest value',
                        type: 'min',
                        valueDim: 'lowest'
                    },
                    {
                        name: 'average value on close',
                        type: 'average',
                        valueDim: 'close'
                    }
                ],
                tooltip: {
                    formatter: function (param) {
                        return param.name + '<br>' + (param.data.coord || '');
                    }
                }
            },
            markLine: {
                symbol: ['none', 'none'],
                data: [
                    [
                        {
                            name: 'from lowest to highest',
                            type: 'min',
                            valueDim: 'lowest',
                            symbol: 'circle',
                            symbolSize: 10,
                            label: {
                                normal: {show: false},
                                emphasis: {show: false}
                            }
                        },
                        {
                            type: 'max',
                            valueDim: 'highest',
                            symbol: 'circle',
                            symbolSize: 10,
                            label: {
                                normal: {show: false},
                                emphasis: {show: false}
                            }
                        }
                    ],
                    {
                        name: 'min line on close',
                        type: 'min',
                        valueDim: 'close'
                    },
                    {
                        name: 'max line on close',
                        type: 'max',
                        valueDim: 'close'
                    }
                ]
            }
        },
        {
            name: 'MA5',
            type: 'line',
            data: calculateMA(5),
            smooth: true,
            lineStyle: {
                normal: {opacity: 0.5}
            }
        },
        {
            name: 'MA10',
            type: 'line',
            data: calculateMA(10),
            smooth: true,
            lineStyle: {
                normal: {opacity: 0.5}
            }
        },
        {
            name: 'MA20',
            type: 'line',
            data: calculateMA(20),
            smooth: true,
            lineStyle: {
                normal: {opacity: 0.5}
            }
        },
        {
            name: 'MA30',
            type: 'line',
            data: calculateMA(30),
            smooth: true,
            lineStyle: {
                normal: {opacity: 0.5}
            }
        },

    ]
};
myChart.setOption(option);
    });
</script>
"""

In [26]:
HTML(datahead + datavar + funcstr + namestr + functail)